# Loyola Supply Chain Data Analysis Competition

## Contributers:

    Aarij Rehman
    Charlie Marshall

## Loading Packages

In [46]:
import pandas as pd
import numpy as np
import datetime as dt

## Loading in Data

For Charlie's Directory

In [4]:
FreightWaves=pd.read_excel("/Users/charlesmarshall/Desktop/Loyola-Supply-Chain-Competition/FreightWaves External Data.xlsx")

In [49]:
ShipperData=pd.read_excel("/Users/charlesmarshall/Desktop/Loyola-Supply-Chain-Competition/Loyola 2nd Annual Data Competition - Shipper Data Set (Trailing 2-Years).xlsx")

For Aarij's Directory

In [7]:
FreightWaves=pd.read_excel("/Users/aarij/Desktop/python-projects/loyola-supply-chain/FreightWaves External Data.xlsx")

In [6]:
ShipperData=pd.read_excel("/Users/aarij/Desktop/python-projects/loyola-supply-chain/Loyola 2nd Annual Data Competition - Shipper Data Set (Trailing 2-Years).xlsx")

In [8]:
ShipperData.head()

,Load Number,Book Date,Pickup Date,Delivery Date,Origin Market,3-Digit Origin #,Origin State,Dest Market,3-Digit Dest #,Dest State,Mode,Mileage,Spot/Contract,Revenue,Accessorial Cost,Load Type
0,1046,2018-02-26,2018-02-26,2018-02-27,Atlanta,300,GA,NaN,217,MD,Temp Control,1316,Spot,1650.00,0,Ad-hoc
1,1223,2018-02-15,2018-02-16,2018-02-18,Chicago,463,IN,NaN,820,WY,Temp Control,1920,Spot,3480.00,0,Ad-hoc
2,1280,2018-02-15,2018-02-15,2018-02-19,Los Angeles,906,CA,NaN,554,MN,Dry Van,3816,Spot,3567.80,0,Ad-hoc
3,1282,2018-02-15,2018-02-16,2018-02-20,Chicago,601,IL,Chicago,606,IL,Dry Van,48,Spot,203.35,-42,Ad-hoc
4,1340,2018-02-16,2018-02-19,2018-02-19,Chicago,606,IL,Chicago,607,IL,Dry Van,42,Spot,343.88,0,Dedicated


## EDA

Understanding the data:

In [9]:
ShipperData['Book Date']=pd.to_datetime(ShipperData['Book Date'])
ShipperData['Pickup Date']=pd.to_datetime(ShipperData['Pickup Date'])
ShipperData['Delivery Date']=pd.to_datetime(ShipperData['Delivery Date'])

In [10]:
set(ShipperData['Origin Market'])

{'Atlanta', 'Chicago', 'Dallas', 'Los Angeles', 'Philadelphia', 'Seattle'}

In [54]:
set(ShipperData['Dest Market'])

{'Atlanta', 'Chicago', 'Dallas', 'Los Angeles', 'Philadelphia', 'Seattle', nan}

In [11]:
set(ShipperData['Mode'])

{'Air',
 'Drayage',
 'Dry Van',
 'FlatBed',
 'Intermodal',
 'Ocean',
 'Other',
 'Reefer',
 'Temp Control',
 'Truck'}

In [12]:
set(ShipperData['Spot/Contract'])

{'Contract', 'Spot'}

In [13]:
set(ShipperData['Load Type'])

{'Ad-hoc', 'Consistent', 'Critical', 'Dedicated', 'Standard', 'White Glove'}

## Preprocessing

Some preproccessing that needs to be done to the whole data set

### 7 FreightWaves Futures Lanes:
    VLS (LA -> Seattle)
    VSL (Seattle -> LA)
    VLD (LA -> Dallas)
    VDL (Dallas -> LA)
    VCA (Chicago -> Atlanta)
    VAP (Atlanta -> Philadelphia)
    VPC (Philadelphia -> Chicago)
    
    If the load is not delivered in one of these lanes, it will be marked 'Other'

## CHARLIE Q1 is not defined at this point

In [14]:
#Q1[ and Q1]
((Q1['Origin Market']=='Los Angeles') & (Q1['Dest Market']=='Seattle'))[4]

NameError: name 'Q1' is not defined

In [92]:
# Categorizing the Future Lanes
Lane = []
org = list(ShipperData['Origin Market'])
dst = list(ShipperData['Dest Market'])
los = 'Los Angeles'
sea = 'Seattle'
dal = 'Dallas'
chi = 'Chicago'
atl = 'Atlanta'
phi = 'Philadelphia'

futures = {}
futures[(los, sea)] = 'VLS'
futures[(sea, los)] = 'VSL'
futures[(los, dal)] = 'VLD'
futures[(dal, los)] = 'VDL'
futures[(chi, atl)] = 'VCA'
futures[(atl, phi)] = 'VAP'
futures[(phi, chi)] = 'VPC'

In [93]:
Lane=[]
for i in range(len(org)):
    if (org[i], dst[i]) in futures:
        Lane.append(futures[(org[i], dst[i])])
    else:
        Lane.append('Other')

In [94]:
ShipperData['Lane'] = Lane

In [95]:
ShipperData.head()

,Load Number,Book Date,Pickup Date,Delivery Date,Origin Market,3-Digit Origin #,Origin State,Dest Market,3-Digit Dest #,Dest State,Mode,Mileage,Spot/Contract,Revenue,Accessorial Cost,Load Type,Lane
0,1046,2018-02-26,2018-02-26,2018-02-27,Atlanta,300,GA,NaN,217,MD,Temp Control,1316,Spot,1650.00,0,Ad-hoc,Other
1,1223,2018-02-15,2018-02-16,2018-02-18,Chicago,463,IN,NaN,820,WY,Temp Control,1920,Spot,3480.00,0,Ad-hoc,Other
2,1280,2018-02-15,2018-02-15,2018-02-19,Los Angeles,906,CA,NaN,554,MN,Dry Van,3816,Spot,3567.80,0,Ad-hoc,Other
3,1282,2018-02-15,2018-02-16,2018-02-20,Chicago,601,IL,Chicago,606,IL,Dry Van,48,Spot,203.35,-42,Ad-hoc,Other
4,1340,2018-02-16,2018-02-19,2018-02-19,Chicago,606,IL,Chicago,607,IL,Dry Van,42,Spot,343.88,0,Dedicated,Other


## Questions to Ask:

#### Part 1    
    1) Does 2019 mean the book, pickup, or delivery date is in 2019? There are many times when the one of these three meaures are in another year while other measures are in 2019.
    2) When you say "Truckload" RPM", does this mean only for trucks and dry vans?
    3) How should we deal with instances where the mileage is 0 or blank? Outlier
    4) How would you define "volatility"? Does it have something to do with spot vs contract market prices and how they change? Maybe the spread of the prices?

## Question 1

    Calculating overall average Truckload rate per mile in 2019?  Hint: rate per mile is total revenue for a move divided by total mileage.  Please also note that “Dry Van” and “Truck” mean the same thing here:

    Steps:
    Year = 2019
    Truckload = "Dry Van" or "Truck" in Mode ??? NOT SURE IF "TRUCKLOAD" MEANS JUST THESE TWO ANYMORE ???
    Create column of Rate per mile (RPM), divide revenue by mileage for each truckload
    Sum the RPM column, divide by number of instances to find AVERAGE
    

### Converting all date columns to datetime

In [79]:
Shipper2019=ShipperData[ShipperData['Pickup Date'].dt.year == 2019].copy()

In [324]:
Q1=Shipper2019[Shipper2019['Mode'].isin(['Dry Van','Truck'])]

In [325]:
Q1=Q1.reset_index(drop=True)

In [244]:
Q1[Q1['Mileage']==0]

,Load Number,Book Date,Pickup Date,Delivery Date,Origin Market,3-Digit Origin #,Origin State,Dest Market,3-Digit Dest #,Dest State,Mode,Mileage,Spot/Contract,Revenue,Accessorial Cost,Load Type,Lane
39,155407,2019-01-04,2019-01-04,2019-01-07,Chicago,608,IL,Chicago,608,IL,Dry Van,0,Spot,250.54,-5,Dedicated,Other
61,138663,2019-01-02,2019-01-02,2019-01-31,Chicago,605,IL,Chicago,605,IL,Dry Van,0,Spot,450.00,450,Ad-hoc,Other
62,138664,2019-01-02,2019-01-02,2019-01-31,Chicago,605,IL,Chicago,605,IL,Dry Van,0,Spot,450.00,450,Ad-hoc,Other
64,141319,2019-01-03,2019-01-06,2019-02-06,Chicago,605,IL,Chicago,605,IL,Dry Van,0,Spot,575.00,575,Ad-hoc,Other
65,141320,2019-01-03,2019-01-06,2019-02-06,Chicago,605,IL,Chicago,605,IL,Dry Van,0,Spot,450.00,450,Ad-hoc,Other
66,142784,2019-01-08,2019-01-08,2019-02-08,Chicago,605,IL,Chicago,605,IL,Dry Van,0,Spot,450.00,450,Ad-hoc,Other
146,150415,2019-01-22,2019-01-22,2019-02-22,Chicago,605,IL,Chicago,605,IL,Dry Van,0,Spot,450.00,450,Ad-hoc,Other
147,150423,2019-01-22,2019-01-22,2019-02-22,Chicago,605,IL,Chicago,605,IL,Dry Van,0,Spot,450.00,450,Ad-hoc,Other
148,150428,2019-01-22,2019-01-22,2019-02-22,Chicago,605,IL,Chicago,605,IL,Dry Van,0,Spot,450.00,450,Ad-hoc,Other
223,152133,2019-01-24,2019-01-27,2019-02-27,Chicago,605,IL,Chicago,605,IL,Dry Van,0,Spot,450.00,450,Ad-hoc,Other


In [245]:
Q1[Q1['Revenue']==0]

,Load Number,Book Date,Pickup Date,Delivery Date,Origin Market,3-Digit Origin #,Origin State,Dest Market,3-Digit Dest #,Dest State,Mode,Mileage,Spot/Contract,Revenue,Accessorial Cost,Load Type,Lane
240,152630,2019-03-04,2019-01-02,2019-01-03,Los Angeles,917,CA,NaN,953,CA,Dry Van,(blank),Spot,0.0,0,Ad-hoc,Other
421,154220,2019-01-02,2019-01-02,2019-01-02,Chicago,600,IL,Chicago,601,IL,Dry Van,44,Spot,0.0,0,Ad-hoc,Other
422,154222,2019-01-02,2019-01-03,2019-01-03,Chicago,600,IL,Chicago,601,IL,Dry Van,44,Spot,0.0,0,Ad-hoc,Other
423,154224,2019-01-02,2019-01-04,2019-01-04,Chicago,600,IL,Chicago,601,IL,Dry Van,44,Spot,0.0,0,Ad-hoc,Other
425,154256,2019-01-02,2019-01-02,2019-01-02,Chicago,601,IL,Chicago,601,IL,Dry Van,0,Spot,0.0,-1,Ad-hoc,Other
844,156401,2019-01-07,2019-01-07,2019-01-07,Chicago,600,IL,Chicago,601,IL,Dry Van,44,Spot,0.0,0,Ad-hoc,Other
845,156404,2019-01-07,2019-01-08,2019-01-08,Chicago,600,IL,Chicago,601,IL,Dry Van,44,Spot,0.0,0,Ad-hoc,Other
846,156406,2019-01-07,2019-01-09,2019-01-09,Chicago,600,IL,Chicago,601,IL,Dry Van,44,Spot,0.0,0,Ad-hoc,Other
847,156407,2019-01-07,2019-01-10,2019-01-10,Chicago,600,IL,Chicago,601,IL,Dry Van,44,Spot,0.0,0,Ad-hoc,Other
848,156408,2019-01-07,2019-01-11,2019-01-11,Chicago,600,IL,Chicago,601,IL,Dry Van,44,Spot,0.0,0,Ad-hoc,Other


In [246]:
Q1['Revenue'][43095]/Q1['Mileage'][43095]

/Users/charlesmarshall/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.


nan

In [248]:
rpm=np.empty(len(Q1))

In [249]:
for i in range(len(Q1)):
    if Q1['Mileage'][i]==0:
        rpm[i]=12345
    else:
        try:
            rpm[i]= Q1['Revenue'][i]/Q1['Mileage'][i]
        except TypeError:
            rpm[i]=54321

In [250]:
# 762 (out of 43,000) instances where the revenue is 0
len(rpm[rpm==12345])

762

In [251]:
for i in range(len(Q1)):
    if Q1['Mileage'][i]==0:
        rpm[i]=0
    else:
        try:
            rpm[i]= Q1['Revenue'][i]/Q1['Mileage'][i]
        except TypeError:
            rpm[i]=0

In [252]:
Q1['RPM'] = rpm

In [254]:
Average2019RPM=sum(Q1['RPM'])/len(Q1)

In [255]:
Average2019RPM

3.605813503312686

### Question 1a

    Using this Redwood rate per mile data, rank the 7 FreightWaves Futures lanes (page 2 at link) by rate volatility for full year 2019.
    
    Volatility = sd
    So, the Futures lanes will be ranked by sd of RPM data. Highest to lowest.

In [286]:
LaneVolitility2019=pd.DataFrame(Q1.groupby('Lane')['RPM'].std(),columns=['RPM']).drop(['Other']).sort_values(by=['RPM'],ascending=False)

In [287]:
LaneVolitility2019

,RPM
Lane,
VPC,5.410751
VCA,1.275223
VLS,0.789660
VAP,0.625256
VLD,0.552769
VDL,0.357352
VSL,0.296679


### Question 1b

    Rank these lanes by total Redwood Truckload volume for full year 2019.
    
    Defining "volume" as the number (count) of truckloads delivered in each lane in 2019.

In [318]:
LaneVolume2019=pd.DataFrame(Q1.groupby('Lane')['RPM'].count()).drop(['Other']).sort_values(by=['RPM'],ascending=False).rename(columns={'RPM':'Volume'})

In [319]:
LaneVolume2019

,Volume
Lane,
VCA,396
VSL,275
VDL,198
VPC,197
VLS,174
VLD,107
VAP,40


## Question 2

    Rank the lanes provided in the FreightWaves data by total intermodal volume for full year 2019.

In [322]:
Q2=Shipper2019[Shipper2019['Mode'].isin(['Intermodal'])]

In [326]:
Q2=Q2.reset_index(drop=True)

In [335]:
IntermodalLaneVolume2019=pd.DataFrame(Q2.groupby('Lane')['Load Number'].count()).drop(['Other']).sort_values(by=['Load Number'],ascending=False).rename(columns={'RPM':'Volume'})

In [336]:
IntermodalLaneVolume2019

,Load Number
Lane,
VDL,47
VLD,29
VCA,1


## Question 3

    Graph monthly volatility in fuel prices by markets provided in the FreightWaves data.  Do any markets stand out to you in particular?

In [337]:
FreightWaves

,ticker,granularity1,symbol,data_timestamp,data_value,index_name,Description
0,COSP,RES,COSP.RES,2019-04-30,512113.00,"Total Construction Spending (000,000s)",Residential
1,COSP,RES,COSP.RES,2019-09-30,529455.00,"Total Construction Spending (000,000s)",Residential
2,COSP,RES,COSP.RES,2019-11-30,540457.00,"Total Construction Spending (000,000s)",Residential
3,COSP,RES,COSP.RES,2018-02-28,577315.00,"Total Construction Spending (000,000s)",Residential
4,COSP,RES,COSP.RES,2018-12-31,517951.00,"Total Construction Spending (000,000s)",Residential
5,COSP,RES,COSP.RES,2019-12-31,548027.00,"Total Construction Spending (000,000s)",Residential
6,COSP,RES,COSP.RES,2018-09-30,536729.00,"Total Construction Spending (000,000s)",Residential
7,COSP,RES,COSP.RES,2018-11-30,527986.00,"Total Construction Spending (000,000s)",Residential
8,COSP,RES,COSP.RES,2019-01-31,514893.00,"Total Construction Spending (000,000s)",Residential
9,COSP,RES,COSP.RES,2019-08-31,525229.00,"Total Construction Spending (000,000s)",Residential
